## Spark Data Wrangling Exercises

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [55]:
spark = SparkSession.builder.getOrCreate()

### 1) Read the case, department, and source data into their own spark dataframes.

In [56]:
dept = spark.read.csv('dept.csv', header=True, inferSchema=True)

In [57]:
df = spark.read.csv("case.csv", sep = ',', header=True, inferSchema=True)

In [58]:
source = spark.read.csv('source.csv', header=True, inferSchema=True)

### 2) Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`
- Inspect your folder structure. What do you notice?

In [59]:
source.write.format('csv').mode('overwrite').option('header', 'true').save('sources_csv')
source.write.format('json').mode('overwrite').option('header', 'true').save('sources_json')

Woah, it made two new directories with the associated files.

### 3) Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [60]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [61]:
dept.show(2, vertical=True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
only showing top 2 rows



In [62]:
dept.groupBy('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|                YES|   31|
|                 NO|    8|
+-------------------+-----+



In [63]:
dept = dept.withColumn('dept_subject_to_SLA', col('dept_subject_to_SLA') == 'Yes')

In [64]:
dept.show(2, vertical=True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | false                
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | false                
only showing top 2 rows



In [65]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'boolean')]

In [66]:
df.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [67]:
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [68]:
# Turn values to bools
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

In [69]:
# Turn district to string
df = df.withColumn("council_district", col("council_district").cast("string"))

In [70]:
# Edit format of the address
df = df.withColumn("request_address", trim(lower(df.request_address)))

In [71]:
# turn days late to weeks 
df = df.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

In [72]:
# change dates to timestamp
fmt = "M/d/yy H:mm"
df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("SLA_due_date", to_timestamp("case_opened_date", fmt))
)

In [73]:
max_date = df.agg(max(df.case_closed_date))

In [74]:
df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 00:42:00|     890|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-01 00:46:00|     890|             0|            0|
|       true|2018-01-01 00:48:00|2018-01-01 00:48:00|     890|             0|            0|
|       true|2018-01-01 01:29:00|2018-01-01 01:29:00|     890|             0|            0|
|       true|2018-01-01 01:34:00|2018-01-01 01:34:00|     890|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|ca

In [75]:
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('num_weeks_late', 'double'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

In [76]:
df = (
    df
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [77]:
source.show(2, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
only showing top 2 rows



In [78]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

______________________________________________________________________________________

### 1) How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [79]:
(
    df.select(df.SLA_due_date, df.case_opened_date, df.case_closed_date, df.case_closed)
    .filter(df.case_closed == "false")
    .sort("case_opened_date")
).first()

Row(SLA_due_date=datetime.datetime(2017, 1, 1, 13, 48), case_opened_date=datetime.datetime(2017, 1, 1, 13, 48), case_closed_date=datetime.datetime(2017, 1, 1, 13, 48), case_closed=False)

In [80]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 890                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | false                
only showing top 1 row



In [ ]:
#Using SQL query type language for this one

# How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

### 2) How many Stray Animal cases are there?

In [81]:
df.filter(df.service_request_type == "Stray Animal").count()

26760

### 3) How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [83]:
(
    df.filter(df.dept_division == "Field Operations")
    .filter(df.service_request_type != "Officer Standby")
    .count()
)

113902

In [85]:
# Another way to do it
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

113902

### 5) Extract the year from the case_closed_date column.

In [86]:
(
    df.withColumn("year", year("case_closed_date"))
    .select("year")
    .show(10)
)

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 10 rows



### 6) Convert num_days_late from days to hours in new columns num_hours_late.

In [87]:
# turn days late to weeks 
df = df.withColumn(
    "num_hours_late", expr("num_days_late * 24 AS num_hours_late")
)

### 7) Join the case data with the source and department data.

In [88]:
source.show(1)


+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [89]:
df.select("source_id").show(10)

+---------+
|source_id|
+---------+
| svcCRMLS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
+---------+
only showing top 10 rows



In [90]:
df = df.join(source, "source_id", "left")

In [91]:
df.show(2, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 890                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | false                
 num_hours_late       | -23964.210

### 8) Are there any cases that do not have a request source?

In [92]:
df.filter(col("source_id").isNull()).show(vertical=True)

(0 rows)



In [93]:
df.filter(col("source_username").isNull()).show(vertical=True)

(0 rows)



### 9) What are the top 10 service request types in terms of number of requests?

In [95]:
df.groupby('service_request_type').count().sort(col('count').desc()).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### 10) What are the top 10 service request types in terms of average days late?

In [99]:
(
    df.groupby(df.service_request_type)
    .agg(mean(df.num_days_late).alias("mean_days"), count('*').alias('n_cases'))
    .sort(col("mean_days").desc())
    .show(10, truncate = False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |mean_days         |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Junk Yards                    |175.9563621042095 |296    |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Storage of Used Mattress              |142.112556415     |8      |
|Zoning: Recycle Yard                  |135.92851612479797|198    |
|Donation Container Enforcement        |131.75610506358706|155    |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
|Traffic Signal Graffiti               |101.79846062200002|5      |
|Complaint                             |72.87050230311695 |2420   |
+--------------------------------------+------------------+-------+
only showing top 10 rows



In [98]:
(
    df.where('case_late')
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509804|51     |
|Donation Container Enforcement        |171.09115313942618|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### 11) Does number of days late depend on department?

In [100]:
(
    df.groupby(df.department)
    .agg(mean(df.num_days_late).alias("total_days_late"))
    .sort(col("total_days_late").desc())
    .show(10, truncate = False)
)

+------------------------+-------------------+
|department              |total_days_late    |
+------------------------+-------------------+
|Customer Service        |59.737091496300785 |
|Solid Waste             |-2.2000575136721747|
|Metro Health            |-4.911766979607002 |
|Parks & Recreation      |-5.251521960055133 |
|Trans & Cap Improvements|-20.612837354052626|
|DSD/Code Enforcement    |-38.36938892614453 |
|Animal Care Services    |-226.51783940550382|
|City Council            |null               |
+------------------------+-------------------+



In [ ]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late').withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

### 12) How do number of days late depend on department and request type?

In [115]:
(
    df.groupby(df.department, df.service_request_type)
    .agg(mean(df.num_days_late).alias("total_days_late"))
    .sort(col("total_days_late").desc())
    .show(10, truncate = False)
)

+------------------------+--------------------------------------+------------------+
|department              |service_request_type                  |total_days_late   |
+------------------------+--------------------------------------+------------------+
|DSD/Code Enforcement    |Zoning: Junk Yards                    |175.9563621042095 |
|DSD/Code Enforcement    |Labeling for Used Mattress            |162.43032902285717|
|DSD/Code Enforcement    |Record Keeping of Used Mattresses     |153.99724039428568|
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr|151.63868055333333|
|DSD/Code Enforcement    |Storage of Used Mattress              |142.112556415     |
|DSD/Code Enforcement    |Zoning: Recycle Yard                  |135.92851612479797|
|DSD/Code Enforcement    |Donation Container Enforcement        |131.75610506358706|
|DSD/Code Enforcement    |License Requied Used Mattress Sales   |128.79828704142858|
|Trans & Cap Improvements|Traffic Signal Graffiti               |

In [ ]:
(df.filter("case_closed").filter("case_late").groupby("standardized_dept_name", "service_request_type").agg(mean("num_days_late")).withColumn("days_late", round(col("days_late"), 1)).sort(desc("days_late")).show(40, truncate=False))

### Dates

In [114]:
max_date = df.select(max('case_opened_date'), max('case_closed_date')).first()[0]
max_date = max_date.strftime('%Y-%m-%d %H:%M:%S')

df = (
    df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)